In [ ]:
# Web scraping
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Data manipulation
import pandas as pd

# Regular expression
import re

In [95]:
# Main link
main_link = 'https://dibi.bnpb.go.id/xdibi?pr=32&kb=01&jn=101&th=&bl=&tb=2&st=3&kf=0&start=0'

In [96]:
# Open the driver
DRIVER_PATH = '../bin/chromedriver'
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Acer\AppData\Local\Temp\ipykernel_26136\774946110.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [97]:
# Access to main link
driver.get(main_link)

In [98]:
# Initialize dictionary
dict_init = {}

In [100]:
# Find the table rows within the dataCollection element
for page in range(1, 20):
    dataCollection = driver.find_element(By.CLASS_NAME, 'col-md-12')
    
    columnCollection = dataCollection.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    
    # Get column names
    listColNames = []
    
    for elemCol in columnCollection:
        listColNames.append(elemCol.text)
    
    # Result
    listCols = [re.sub('[^a-zA-Z\d]', '', x) for x in listColNames]
    listCols[len(listCols) - 1] = 'Links'
    
    # Initialize the dictionary for each page
    dict_page = {key: [] for key in listCols}
    
    # Find the table rows within the dataCollection element
    valueCollection = dataCollection.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    print('Length of data in page {}: {} rows'.format(page, len(valueCollection)))
    
    # Get data
    for row in valueCollection:
        elemValues = row.find_elements(By.TAG_NAME, 'td')
        for col in range(len(elemValues)):
            try:
                value = elemValues[col].text
            except:
                value = None
            # Append values to the page dictionary
            dict_page[list(dict_page.keys())[col]].append(value)
    
    # Get links
    links = driver.find_elements(By.CLASS_NAME, 'btn.btn-info.btn-xs')
    listLink = [link.get_attribute('href') for link in links]
    
    # Get the span detailed
    spanDetailed = []
    for i in dataCollection.find_elements(By.TAG_NAME, 'span'):
        # Get the span type
        spanType = i.get_attribute('data-toggle')
        
        # Select only popover
        if spanType == 'popover':
            classValue = i.get_attribute('data-original-title')
            if re.sub('[^a-zA-Z\d]', '', classValue) in ['Keterangan', 'Korban', 'Kerusakan']:
                spanDetailed.append(i)
    
    listDetailed = []
    for idx in range(3, len(spanDetailed) + 3, 3):
        # Dictionary for storing data
        dictDetailed = {}
        
        # Looping per 3 object
        for j in range(idx - 3, idx):
            classValue = re.sub('[^a-zA-Z\d]', '', spanDetailed[j].get_attribute('data-original-title'))
            # Get the values
            if classValue != None:
                dictDetailed.update(
                    {
                        classValue: spanDetailed[j].get_attribute('data-content')
                    }
                )
            else:
                dictDetailed.update(
                    {
                        classValue: None
                    }
                )
        
        # Append the dictionary
        listDetailed.append(dictDetailed)
    
    # Update the page dictionary
    dict_page.update(
        {
            'Detail': listDetailed,
            'Links': listLink
        }
    )
    
    # Append the page dictionary to the main dictionary
    for key, value in dict_page.items():
        dict_init.setdefault(key, []).extend(value)
    
    # Go to the next page
    next_page_link = driver.find_element(By.CSS_SELECTOR, 'a[data-ci-pagination-page="{}"]'.format(page+1))
    driver.get(next_page_link.get_attribute('href'))

Length of data in page 1: 10 rows
Length of data in page 2: 10 rows
Length of data in page 3: 10 rows
Length of data in page 4: 10 rows
Length of data in page 5: 10 rows
Length of data in page 6: 10 rows
Length of data in page 7: 10 rows
Length of data in page 8: 10 rows
Length of data in page 9: 10 rows
Length of data in page 10: 10 rows
Length of data in page 11: 10 rows
Length of data in page 12: 10 rows
Length of data in page 13: 10 rows
Length of data in page 14: 10 rows
Length of data in page 15: 10 rows
Length of data in page 16: 10 rows
Length of data in page 17: 10 rows
Length of data in page 18: 10 rows
Length of data in page 19: 10 rows


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[data-ci-pagination-page="20"]"}
  (Session info: chrome=114.0.5735.90)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D1A813+48355]
	(No symbol) [0x00CAC4B1]
	(No symbol) [0x00BB5358]
	(No symbol) [0x00BE09A5]
	(No symbol) [0x00BE0B3B]
	(No symbol) [0x00C0E232]
	(No symbol) [0x00BFA784]
	(No symbol) [0x00C0C922]
	(No symbol) [0x00BFA536]
	(No symbol) [0x00BD82DC]
	(No symbol) [0x00BD93DD]
	GetHandleVerifier [0x00F7AABD+2539405]
	GetHandleVerifier [0x00FBA78F+2800735]
	GetHandleVerifier [0x00FB456C+2775612]
	GetHandleVerifier [0x00DA51E0+616112]
	(No symbol) [0x00CB5F8C]
	(No symbol) [0x00CB2328]
	(No symbol) [0x00CB240B]
	(No symbol) [0x00CA4FF7]
	BaseThreadInitThunk [0x76067D59+25]
	RtlInitializeExceptionChain [0x77C1B74B+107]
	RtlClearBits [0x77C1B6CF+191]


In [91]:
# Convert dict_init to DataFrame
df = pd.DataFrame.from_dict(dict_init)

In [92]:
# Export DataFrame to CSV
df.to_csv('data.csv', index=False)

In [101]:
# Main link
main_link_2 = 'https://dibi.bnpb.go.id/xdibi?pr=32&kb=01&jn=106&th=&bl=&tb=2&st=3&kf=0&start=0&start=0'

In [102]:
# Access to main link
driver.get(main_link_2)

In [103]:
# Initialize dictionary
dict_init_2 = {}

In [104]:
# Find the table rows within the dataCollection element
for page in range(1, 20):
    dataCollection = driver.find_element(By.CLASS_NAME, 'col-md-12')
    
    columnCollection = dataCollection.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    
    # Get column names
    listColNames = []
    
    for elemCol in columnCollection:
        listColNames.append(elemCol.text)
    
    # Result
    listCols = [re.sub('[^a-zA-Z\d]', '', x) for x in listColNames]
    listCols[len(listCols) - 1] = 'Links'
    
    # Initialize the dictionary for each page
    dict_page = {key: [] for key in listCols}
    
    # Find the table rows within the dataCollection element
    valueCollection = dataCollection.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    print('Length of data in page {}: {} rows'.format(page, len(valueCollection)))
    
    # Get data
    for row in valueCollection:
        elemValues = row.find_elements(By.TAG_NAME, 'td')
        for col in range(len(elemValues)):
            try:
                value = elemValues[col].text
            except:
                value = None
            # Append values to the page dictionary
            dict_page[list(dict_page.keys())[col]].append(value)
    
    # Get links
    links = driver.find_elements(By.CLASS_NAME, 'btn.btn-info.btn-xs')
    listLink = [link.get_attribute('href') for link in links]
    
    # Get the span detailed
    spanDetailed = []
    for i in dataCollection.find_elements(By.TAG_NAME, 'span'):
        # Get the span type
        spanType = i.get_attribute('data-toggle')
        
        # Select only popover
        if spanType == 'popover':
            classValue = i.get_attribute('data-original-title')
            if re.sub('[^a-zA-Z\d]', '', classValue) in ['Keterangan', 'Korban', 'Kerusakan']:
                spanDetailed.append(i)
    
    listDetailed = []
    for idx in range(3, len(spanDetailed) + 3, 3):
        # Dictionary for storing data
        dictDetailed = {}
        
        # Looping per 3 object
        for j in range(idx - 3, idx):
            classValue = re.sub('[^a-zA-Z\d]', '', spanDetailed[j].get_attribute('data-original-title'))
            # Get the values
            if classValue != None:
                dictDetailed.update(
                    {
                        classValue: spanDetailed[j].get_attribute('data-content')
                    }
                )
            else:
                dictDetailed.update(
                    {
                        classValue: None
                    }
                )
        
        # Append the dictionary
        listDetailed.append(dictDetailed)
    
    # Update the page dictionary
    dict_page.update(
        {
            'Detail': listDetailed,
            'Links': listLink
        }
    )
    
    # Append the page dictionary to the main dictionary
    for key, value in dict_page.items():
        dict_init_2.setdefault(key, []).extend(value)
    
    # Go to the next page
    next_page_link = driver.find_element(By.CSS_SELECTOR, 'a[data-ci-pagination-page="{}"]'.format(page+1))
    driver.get(next_page_link.get_attribute('href'))

Length of data in page 1: 10 rows
Length of data in page 2: 10 rows
Length of data in page 3: 10 rows
Length of data in page 4: 1 rows


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[data-ci-pagination-page="5"]"}
  (Session info: chrome=114.0.5735.90)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D1A813+48355]
	(No symbol) [0x00CAC4B1]
	(No symbol) [0x00BB5358]
	(No symbol) [0x00BE09A5]
	(No symbol) [0x00BE0B3B]
	(No symbol) [0x00C0E232]
	(No symbol) [0x00BFA784]
	(No symbol) [0x00C0C922]
	(No symbol) [0x00BFA536]
	(No symbol) [0x00BD82DC]
	(No symbol) [0x00BD93DD]
	GetHandleVerifier [0x00F7AABD+2539405]
	GetHandleVerifier [0x00FBA78F+2800735]
	GetHandleVerifier [0x00FB456C+2775612]
	GetHandleVerifier [0x00DA51E0+616112]
	(No symbol) [0x00CB5F8C]
	(No symbol) [0x00CB2328]
	(No symbol) [0x00CB240B]
	(No symbol) [0x00CA4FF7]
	BaseThreadInitThunk [0x76067D59+25]
	RtlInitializeExceptionChain [0x77C1B74B+107]
	RtlClearBits [0x77C1B6CF+191]


In [105]:
# Convert dict_init to DataFrame
df = pd.DataFrame.from_dict(dict_init_2)

In [106]:
# Export DataFrame to CSV
df.to_csv('data2.csv', index=False)

In [108]:
# Load data from CSV
df = pd.read_csv('data_banjir.csv')

# Convert DataFrame to Excel
df.to_excel('data_banjir.xlsx', index=False)